## Code Snippet
From bayeslite/metamodel.py

### Done:
* TrollMetamodel

### Next Step:
* NIGnormal
* Dirichlet-Multinomial
* Constrained NIGnormal

## Preamble

In [ ]:
from abc import ABCMeta, abstractmethod

def check_data(data):
    """Check whether data is of proper format (numpy array for now)"""
    print("Data check has not been implemented yet")
    
def check_typespec_dict(typespec):
    """Check whether typespec is a dict mapping variable names to type specs."""
    print("Typespec check has not been implemented yet")

## MyMetamodel
Abstract Base Class for implementing metamodels

In [19]:
from abc import ABCMeta, abstractmethod

def check_data(data):
    """Check whether data is of proper format (numpy array for now)"""
    print("Data check has not been implemented yet")
    
def check_typespec_dict(typespec):
    """Check whether typespec is a dict mapping variable names to type specs."""
    print("Typespec check has not been implemented yet")


class MyMetamodel(object):
    """Metamodel interface.
    
    Subclasses of :class:'MyMetamodel' implement the functionality needed to
    sample from and inquire about the posterior distribution of a generative
    model conditioned on data in a table.
    """
    __metaclass__ = ABCMeta
    
    @abstractmethod
    def __init__ (self, data, typespec):
        check_data(data)
        check_typespec_dict(typespec)
        
        self.typespec = typespec
        self.data = data
        
    @abstractmethod
    def name(self):
        """Return the name of the metamodel as a str"""
        raise NotImplementedError
    
    @abstractmethod
    def infer(self):
        """Predict a value for each missing value."""
        raise NotImplementedError
    
    @abstractmethod
    def predict(self, colno, threshold, numsamples=None):
        """Predict a value for a column, if confidence is high enough."""
        value, confidence = self.predict_confidence(self, colno, numsamples=numsamples)
        if confidence < threshold:
            return float('nan')
        return value
    
    @abstractmethod
    def predict_confidence(self, colno, numsamples=None):
        """Predict a value for a column and return confidence"""
        raise NotImplementedError
        
    @abstractmethod    
    def simulate(self, colnos, constraints, numpredictions=1):
        """Simulate 'colnos' from a generator, subject to 'constraints'.
        
        Returns a list of rows with values for the specified columns.
        
        'colnos'         - list of column numbers.
        'constraints'    - list of ''(colno, value)'' pairs.
        'numpredictions' - number of results to return.
        """ 
        raise NotImplementedError
    
    @abstractmethod
    def estimate(self, colno0, colno1):
        """Compute Dependence Probability of <col0> with <col1>"""
        raise NotImplementedError
    
    @abstractmethod
    def logpdf(self, target, constraints):
        """Compute (predictive?) logpdf.
        
                'constraints'    - list of ''(colno, value)'' pairs.
                'targets'        - list of ''(colno, value)'' pairs.
        """
        raise NotImplementedError
        
    @abstractmethod
    def column_mutual_information(self, colno0, colno1, numsamples=100):
        """Compute ``MUTUAL INFORMATION OF <col0> WITH <col1>``."""
        raise NotImplementedError

## TrollMetamodel
Simplest possible model to check whether the structure works.

In [20]:
class TrollMetamodel(MyMetamodel):
    """Troll metamodel for MyMetamodel.

    The metamodel is named ``troll_rng``.
    
    Class Methods:
        __init__(self,data,typespec)
        name(self)
        infer(self, threshold, numsamples=None)
        predict(self, colno, threshold, numsamples=None)
        predict_confidence(self, colno, numsamples=None)
        simulate(self, colnos, constraints, numpredictions=1)
        estimate(self, colno0, colno1)
        column_mutual_information(self, colno0, colno1, numsamples=None)
        logpdf(self, targets, constraints)
    """

    def __init__(self,data,typespec): 
        super(TrollMetamodel, self).__init__(data,typespec)
    
    def name(self): return 'troll_rng'
  
    def infer(self, threshold=1, numsamples=None):
        nanrows, nancols = np.where(np.isnan(self.data))
        for i in np.arange(nancols.size):
            prediction = self.predict(nancols[i], threshold, numsamples)
            self.data[nanrows[i],nancols[i]] = prediction
    
    def predict(self, colno, threshold, numsamples=None):
        super(TrollMetamodel,self).predict(colno,threshold,numsamples=None)
    
    def predict_confidence(self, colno, numsamples=None):
        value = 9.
        confidence = 1.
        return value, confidence
        
    def simulate(self, colnos, constraints=[], numpredictions=1):
        # check_col(colnos)
        # check_col([i for i,_ in constraints])
        for (_, value) in constraints:
            if not value == 9:
                return float("nan")
        header = [colnos]
        X = np.array([[9. for _ in colnos] for _ in np.arange(numpredictions)])
        return header, X
    
    def estimate(self, colno0, colno1):
        # check_col(colno0, colno1)
        dep_prob = 0
        return dep_prob
    
    def column_mutual_information(self, colno0, colno1, numsamples=None):
        mutual_info = 0   
        return mutual_info
    
    def logpdf(self, targets, constraints):
        for (_, value) in constraints:
            if not value == 9:
                return float("nan")
        for (_, value) in targets:
            if not value == 9:
                return float("-inf")
            
        '''Recheck this later'''
        return 0
    
    '''TODO: Rewrite or import docstrings'''


In [21]:
# import numpy as np
# from TrollMetamodel import *

X = np.tile(9., [10,5])
for [i,j] in [[0,0],[0,2],[1,3],[3,4],[4,4],[5,1]]:
    X[i][j] = float('nan')
    
troll = TrollMetamodel(X,'standard')

Data check has not been implemented yet
Typespec check has not been implemented yet


In [38]:
a,b= [float('inf') ,float('inf')]
(a +
     b)

inf

## NIGNormalMetamodel
Metamodel for independent numerical variables, using a Normal-Inverse-Gamma prior for each column.

In [ ]:
targets = [(1,2),(3,5),(7,8)]
target_colnos = np.array([col for (col,_) in targets])

In [ ]:
X = dict('M':1)

In [ ]:
import math

default_data_parameters = dict(
    multinomial=dict(weights=[1.0/5.0]*5),
    continuous=dict(mu=0.0, rho=1.0),
    cyclic=dict(mu=math.pi, kappa=2.0)
    )

default_data_parameters